In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


In [ ]:

file_path = "/content/posture_data.csv"
data = pd.read_csv(file_path)


X = data.iloc[:, :-1]
y = data.iloc[:, -1]


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)



X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Classifier (SVC)": SVC(kernel='linear'),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
}


In [ ]:
results = {}
for name, model in classifiers.items():
    print(f"\nModel: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"Accuracy: {accuracy:.2f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))



Model: Random Forest
Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

     Sitting       1.00      1.00      1.00       149
    Standing       1.00      1.00      1.00       145

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294


Model: Logistic Regression
Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

     Sitting       1.00      1.00      1.00       149
    Standing       1.00      1.00      1.00       145

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294


Model: K-Nearest Neighbors
Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

     Sitting       1.00      1.00      1.00       149
    Standing       1.00      1.00      1.0

In [ ]:
print("\nFinal Model Comparison:")
for name, accuracy in results.items():
    print(f"{name}: {accuracy:.2f}")


Final Model Comparison:
Random Forest: 1.00
Logistic Regression: 1.00
K-Nearest Neighbors: 1.00
Support Vector Classifier (SVC): 1.00
Decision Tree: 0.99
Gradient Boosting: 0.99


In [ ]:
import joblib
from google.colab import files

# Save models
def save_models(classifiers, file_names):
    for name, model in classifiers.items():
        file_name = f"{file_names[name]}.joblib"
        joblib.dump(model, file_name)
        print(f"Model '{name}' saved as {file_name}")

# Define classifiers (from the training code)
classifiers = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Classifier (SVC)": SVC(kernel='linear', random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
}

# Train classifiers (add this if not done already)
for name, model in classifiers.items():
    model.fit(X_train, y_train)  # Assuming `X_train` and `y_train` are already defined

# Define filenames for each model
file_names = {
    "Random Forest": "random_forest_model",
    "Logistic Regression": "logistic_regression_model",
    "K-Nearest Neighbors": "knn_model",
    "Support Vector Classifier (SVC)": "svc_model",
    "Decision Tree": "decision_tree_model",
    "Gradient Boosting": "gradient_boosting_model",
}

# Save all models
save_models(classifiers, file_names)

# Download saved models
#for name, file_name in file_names.items():
#    files.download(f"{file_name}.joblib")


Model 'Random Forest' saved as random_forest_model.joblib
Model 'Logistic Regression' saved as logistic_regression_model.joblib
Model 'K-Nearest Neighbors' saved as knn_model.joblib
Model 'Support Vector Classifier (SVC)' saved as svc_model.joblib
Model 'Decision Tree' saved as decision_tree_model.joblib
Model 'Gradient Boosting' saved as gradient_boosting_model.joblib
